<a href="https://colab.research.google.com/github/skagnihotri/urban_sound_classification/blob/master/sound_classification_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data = pd.read_csv('/content/drive/My Drive/urban sound dataset/ content UrbanSound8K metadata UrbanSound8K.csv')

In [40]:
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [0]:
X_train = pd.read_csv('/content/drive/My Drive/urban sound dataset/ content train_data.csv')
X_test = pd.read_csv('/content/drive/My Drive/urban sound dataset/ content test_data.csv')
y_train = pd.read_csv('/content/drive/My Drive/urban sound dataset/ content train_labels.csv')
y_test = pd.read_csv('/content/drive/My Drive/urban sound dataset/ content test_labels.csv')

In [42]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

X_test.shape, y_test.shape

((836, 6960), (836, 1))

In [43]:
X_train = X_train.reshape((X_train.shape[0], 40, 174, 1))
X_test = X_test.reshape((X_test.shape[0], 40, 174, 1))

X_train.shape, X_test.shape

((7894, 40, 174, 1), (836, 40, 174, 1))

# CNN Model

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Input, BatchNormalization

In [0]:
#model
input = Input(shape= (40, 174, 1))